<a href="https://colab.research.google.com/github/Himanshu069/3D-AI/blob/main/Point_Cloud_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install open3d
!pip install plotly
!pip install trimesh

  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.7/399.7 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 48.1 MB/s eta 0:00:00
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.9
    Uninstalling widgetsnbextension-3.6.9:
      Successfully uninstalled widgetsnbextension-3.6.9
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 700.1/700.1 kB 21.8 MB/s eta 0:00:00


In [ ]:
#generating dataset
import trimesh
import random
import os
import numpy as np

# Create a directory to save the generated shapes
output_dir = "random_shapes_dataset"
os.makedirs(output_dir, exist_ok=True)
labels=[]

# Function to generate a random shape and save it as an OBJ file
def generate_random_shape(index):
    # Randomly choose a shape type: 0 = cube, 1 = sphere, 2 = cylinder, 3 = cone
    shape_type = random.choice(["cube", "sphere", "cylinder", "cone"])

    # Generate random parameters for the shape
    scale = random.uniform(0.5, 2.0)  # Random scale
    if shape_type == "cube":
        labels.append(0)
        shape = trimesh.creation.box(extents=(scale, scale, scale))
    elif shape_type == "sphere":
        labels.append(1)
        radius = random.uniform(0.5, 2.0)
        shape = trimesh.creation.icosphere(subdivisions=3, radius=radius)
    elif shape_type == "cylinder":
        labels.append(2)
        radius = random.uniform(0.5, 1.5)
        height = random.uniform(0.5, 2.0)
        shape = trimesh.creation.cylinder(radius=radius, height=height)
    elif shape_type == "cone":
        labels.append(3)
        radius = random.uniform(0.5, 1.5)
        height = random.uniform(1.0, 3.0)
        shape = trimesh.creation.cone(radius=radius, height=height)

    # Apply random rotation and translation
    shape.apply_transform(trimesh.transformations.random_rotation_matrix())
    shape.apply_translation([random.uniform(-5, 5), random.uniform(-5, 5), random.uniform(-5, 5)])

    # Save the shape as an OBJ file
    filename = f"{shape_type}_{index}.obj"
    filepath = os.path.join(output_dir, filename)
    shape.export(filepath)

# Generate N random shapes
N = 10000  # Number of shapes to generate
for i in range(N):
    generate_random_shape(i)

print(f"Generated {N} random shapes in '{output_dir}' directory.")

Generated 10000 random shapes in 'random_shapes_dataset' directory.


In [ ]:
def obj_to_pointcloud(filepath, num_points = 1024):
                      mesh = trimesh.load(filepath)

                      points, _ = trimesh.sample.sample_surface(mesh, num_points)

                      return points

pointclouds=[]


for obj_file in os.listdir(output_dir):
    if obj_file.endswith(".obj"):
      pointcloud = obj_to_pointcloud(os.path.join(output_dir, obj_file))
      pointclouds.append(pointcloud)

#print(pointclouds[990])
pointclouds = np.array(pointclouds)
print(pointclouds)

[[[ 1.5943878  -1.2032045  -0.62895363]
  [ 0.3899302  -1.13557936 -0.69184822]
  [ 0.4831324  -1.25212153 -1.08372109]
  ...
  [ 0.99993556 -0.32781235 -0.78821632]
  [ 0.80941733 -0.45471002 -0.77669995]
  [ 0.84496715 -1.06845342 -1.38959053]]

 [[-0.99878358 -3.5110768  -0.12265742]
  [-2.31844093 -2.58083452  0.9695199 ]
  [-2.27723235 -3.07580153  0.71428869]
  ...
  [-0.75651058 -2.39537159  1.9195057 ]
  [-0.29077899 -2.84563553  0.97943097]
  [-0.61201094 -1.91263664  0.94268152]]

 [[-0.09898313 -4.39235607 -2.47581782]
  [-0.30445674 -4.63592298 -3.4231956 ]
  [-0.08671665 -4.6875461  -3.18481489]
  ...
  [-0.93928174 -4.74308936 -2.36495151]
  [-0.27568209 -5.27616441 -2.30003345]
  [-0.65976441 -5.28320476 -3.13352902]]

 ...

 [[ 0.98749148 -1.81175544  3.68159511]
  [ 1.42545667 -1.25487166  4.5739242 ]
  [ 1.77285409 -1.54218186  3.19866724]
  ...
  [ 2.02115087 -0.88674624  3.64364744]
  [ 1.99630627 -1.85018713  3.45235393]
  [ 1.32841316 -0.94306111  4.39737424]]

 [

In [ ]:
#converting to pytorch
import torch
from torch.utils.data import Dataset, DataLoader
class PointCloudDataset(Dataset):
  def __init__(self,pointclouds,labels):
    self.pointclouds = pointclouds
    self.labels = labels

  def __len__(self):
      return len(self.pointclouds)

  def __getitem__(self,idx):
      return torch.tensor(self.pointclouds[idx], dtype=torch.float32),torch.tensor(self.labels[idx],dtype=torch.long)

dataset = PointCloudDataset(pointclouds,labels)
dataloader = DataLoader(dataset,batch_size=32,shuffle=True)

In [ ]:
#pointnet model
import torch.nn as nn
import torch.nn.functional as F

class PointNet(nn.Module):
    def __init__(self, num_classes=4):
        super(PointNet, self).__init__()
        self.conv1 = nn.Conv1d(3, 64, 1, bias = True)
        self.conv2 = nn.Conv1d(64, 128, 1,bias = True )
        self.conv3 = nn.Conv1d(128, 256, 1,bias = True)
        self.fc1 = nn.Linear(256, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.dropout = nn.Dropout(0.3)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = torch.max(x, 2)[0]  # Global max pooling
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [ ]:
#training the pointnet model
import torch.optim as optim
#parameters
epochs = 50
learning_rate = 0.001
num_classes = 4

#model, loss function and optimizer

model = PointNet(num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

#training loop
for epoch in range(epochs):
  model.train()
  total_loss = 0
  correct = 0
  total = 0

  for data in dataloader:
      points, labels = data
      points = points.transpose(1,2)
      optimizer.zero_grad()

      #forward pass
      outputs = model(points)
      loss = criterion(outputs, labels)

      #backward-pass and optimizatin
      loss.backward()
      optimizer.step()

      total_loss += loss.item()

      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()

  accuracy = 100 * correct / total
  print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss: .4f}, Accuracy: {accuracy: .2f}%")


Epoch 1/50, Loss:  434.7637, Accuracy:  24.80%
Epoch 2/50, Loss:  434.1398, Accuracy:  25.81%
Epoch 3/50, Loss:  433.8906, Accuracy:  25.64%
Epoch 4/50, Loss:  433.8428, Accuracy:  25.53%
Epoch 5/50, Loss:  433.8792, Accuracy:  25.12%
Epoch 6/50, Loss:  433.7008, Accuracy:  26.22%
Epoch 7/50, Loss:  433.6710, Accuracy:  26.58%
Epoch 8/50, Loss:  433.5835, Accuracy:  26.14%
Epoch 9/50, Loss:  433.6131, Accuracy:  26.63%
Epoch 10/50, Loss:  433.4352, Accuracy:  26.42%
Epoch 11/50, Loss:  433.4691, Accuracy:  26.82%
Epoch 12/50, Loss:  433.4819, Accuracy:  26.66%
Epoch 13/50, Loss:  433.3459, Accuracy:  26.36%
Epoch 14/50, Loss:  433.1609, Accuracy:  26.95%
Epoch 15/50, Loss:  433.1759, Accuracy:  26.80%
Epoch 16/50, Loss:  433.2631, Accuracy:  27.10%
Epoch 17/50, Loss:  433.2171, Accuracy:  26.97%
Epoch 18/50, Loss:  432.9498, Accuracy:  27.49%
Epoch 19/50, Loss:  432.9149, Accuracy:  27.46%
Epoch 20/50, Loss:  433.0819, Accuracy:  27.44%
Epoch 21/50, Loss:  432.8211, Accuracy:  27.21%
E

In [ ]:
#evaluation
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for data in dataloader:
        points, labels = data
        points = points.transpose(1,2)
        outputs = model(points)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on the dataset: {accuracy:.2f}%')

Accuracy on the dataset: 31.98%
